<a href="https://colab.research.google.com/github/gg2001/transformer-circuits/blob/master/puzzles/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Mech Interp Puzzle 1: Suspiciously Similar Embeddings in GPT-Neo](https://www.alignmentforum.org/posts/eLNo7b56kQQerCzp2/mech-interp-puzzle-1-suspiciously-similar-embeddings-in-gpt)

In [ ]:
%pip install transformer_lens plotly nbformat

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62

In [ ]:
from transformer_lens import HookedTransformer, FactoredMatrix
import plotly.express as px
import torch

In [ ]:
model = HookedTransformer.from_pretrained("gpt-neo-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Loaded pretrained model gpt-neo-small into HookedTransformer


In [ ]:
# subsample = torch.randperm(model.cfg.d_vocab)[:5000].to(model.cfg.device)
# W_E = model.W_E[subsample]  # Take a random subset of 5,000 for memory reasons

In [ ]:
W_E = model.W_E

In [ ]:
W_E_normed = W_E / W_E.norm(dim=-1, keepdim=True) # [d_vocab, d_model]
cosine_sims = FactoredMatrix(W_E_normed, W_E_normed.T) # [d_vocab, d_vocab]

In [ ]:
cosine_sims_factored = cosine_sims
cosine_sims = cosine_sims_factored.AB

In [ ]:
# px.histogram(
#    cosine_sims.flatten().detach().cpu().numpy(),
#    title="Pairwise cosine sims of embedding",
#)

In [ ]:
# Step 2: Flatten the matrix and get top 100 similarities
top_k = 100
top_vals, top_indices = torch.topk(cosine_sims_top.view(-1), top_k)

# Step 3: Map the flat indices back to 2D indices
top_token1 = top_indices // cosine_sims.size(1)
top_token2 = top_indices % cosine_sims.size(1)

# Step 4: Prepare the top 100 similarity list
top_pairs = [
    {
        'token_0': model.to_string([int(t0.item())]),
        'token1': model.to_string([int(t1.item())]),
        'similarity': float(sim.item())
    }
    for t0, t1, sim in zip(top_token1, top_token2, top_vals)
]

# Step 5: Exclude the diagonal by setting it to +inf for bottom similarities
cosine_sims_bottom = cosine_sims.masked_fill(mask, float('inf'))

# Step 6: Flatten the matrix and get bottom 100 similarities
bottom_vals, bottom_indices = torch.topk(cosine_sims_bottom.view(-1), top_k, largest=False)

# Step 7: Map the flat indices back to 2D indices
bottom_token1 = bottom_indices // cosine_sims.size(1)
bottom_token2 = bottom_indices % cosine_sims.size(1)

# Step 8: Prepare the bottom 100 similarity list
bottom_pairs = [
    {
        'token_0': model.to_string([int(t0.item())]),
        'token1': model.to_string([int(t1.item())]),
        'similarity': float(sim.item())
    }
    for t0, t1, sim in zip(bottom_token1, bottom_token2, bottom_vals)
]

# Optional: Display the results
print("Top 100 Similarities:")
for pair in top_pairs:
    print(pair)

print("\nBottom 100 Similarities:")
for pair in bottom_pairs:
    print(pair)

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.41 GiB. GPU 0 has a total capacity of 39.56 GiB of which 594.81 MiB is free. Process 75239 has 38.97 GiB memory in use. Of the allocated memory 31.40 GiB is allocated by PyTorch, and 7.09 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)